# Neural Networks for Coffee Rosting
# Outline
- [ 1 - Packages ](#1)
- [ 2 - Neural Networks](#2)
  - [ 2.1 Problem Statement](#2.1)
  - [ 2.2 Dataset](#2.2)
  - [ 2.3 Tensorflow Model Implementation](#2.3)
  - [ 2.4 NumPy Model Implementation (Forward Prop in NumPy)](#2.4)
    

## <a name="1"></a>
## 1 - Packages 

- [numpy](https://numpy.org/) is the fundamental package for scientific computing with Python.
- [matplotlib](http://matplotlib.org) is a popular library to plot graphs in Python.
- [tensorflow](https://www.tensorflow.org/) a popular platform for machine learning.

In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
%matplotlib inline


## <a name="2"></a>
## 2 - Neural Networks

<a name="2.1"></a>
### 2.1 Problem Statement
The problem statement is to develop a neural network model that can predict whether a coffee roasting is good or bad based on the temperature and duration of the roasting process. The dataset contains 200 examples, with each example consisting of two features - temperature and duration. The labels are binary, with 1 indicating a good roasting and 0 indicating a bad roasting.

The objective is to develop a model that can accurately classify the examples into good and bad roasting without using traditional classification metrics such as accuracy, precision, and recall. Instead, we will focus on the model's ability to make correct predictions based on the given features and interpret the results based on the model's decision boundary. The model's performance will be evaluated based on its ability to correctly classify new examples and generalize to unseen data.
<a name="2.2"></a>
### 2.2 Dataset


- The `load_data()` function shown below loads the data into variables `X` and `y`
​

- The data set contains 200 training examples of coffee roasting.  


In [29]:
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))
    
    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))


In [30]:
X,Y = load_coffee_data();
print(X.shape, Y.shape)

(200, 2) (200, 1)


### Normalize Data

In [31]:
print(f"Temperature Max, Min pre normalization: {np.max(X[:,0]):0.2f}, {np.min(X[:,0]):0.2f}")
print(f"Duration    Max, Min pre normalization: {np.max(X[:,1]):0.2f}, {np.min(X[:,1]):0.2f}")
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)  # learns mean, variance
Xn = norm_l(X)
print(f"Temperature Max, Min post normalization: {np.max(Xn[:,0]):0.2f}, {np.min(Xn[:,0]):0.2f}")
print(f"Duration    Max, Min post normalization: {np.max(Xn[:,1]):0.2f}, {np.min(Xn[:,1]):0.2f}")

Temperature Max, Min pre normalization: 284.99, 151.32
Duration    Max, Min pre normalization: 15.45, 11.51
Temperature Max, Min post normalization: 1.66, -1.69
Duration    Max, Min post normalization: 1.79, -1.70


Tile/copy  data to increase the training set size and reduce the number of training epochs.

In [32]:
Xt = np.tile(X,(1000,1))
Yt= np.tile(Y,(1000,1))   
print(Xt.shape, Yt.shape)   

(200000, 2) (200000, 1)


<a name="2.3"></a>
### 2.3 Tensorflow Model Implementation

The neural network we will use in this project is shown in the figure below. 
- This has two dense layers with sigmoid activations.
    

In [33]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        tf.keras.Input(shape=(2,)),
        Dense(3, activation='sigmoid', name = 'layer1'),
        Dense(1, activation='sigmoid', name = 'layer2')
     ]
)

In [34]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 1)                 4         
                                                                 
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [35]:
L1_num_params = 2 * 3 + 3   # W1 parameters  + b1 parameters
L2_num_params = 3 * 1 + 1   # W2 parameters  + b2 parameters
print("L1 params = ", L1_num_params, ", L2 params = ", L2_num_params  )

L1 params =  9 , L2 params =  4


In [36]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print(f"W1{W1.shape}:\n", W1, f"\nb1{b1.shape}:", b1)
print(f"W2{W2.shape}:\n", W2, f"\nb2{b2.shape}:", b2)

W1(2, 3):
 [[ 0.8490095  -0.22524029  0.84335744]
 [ 0.4380275   0.94216895  0.20352936]] 
b1(3,): [0. 0. 0.]
W2(3, 1):
 [[-0.11989427]
 [ 1.0561825 ]
 [ 0.13893509]] 
b2(1,): [0.]


In [37]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)

model.fit(
    Xt,Yt,            
    epochs=10,
)

Epoch 1/10
6250/6250 [==============================] - 14s 2ms/step - loss: 0.5215
Epoch 2/10
6250/6250 [==============================] - 12s 2ms/step - loss: 0.5210
Epoch 3/10
6250/6250 [==============================] - 12s 2ms/step - loss: 0.5211
Epoch 4/10
6250/6250 [==============================] - 13s 2ms/step - loss: 0.5211
Epoch 5/10
6250/6250 [==============================] - 13s 2ms/step - loss: 0.5211
Epoch 6/10
6250/6250 [==============================] - 12s 2ms/step - loss: 0.5210
Epoch 7/10
6250/6250 [==============================] - 13s 2ms/step - loss: 0.5211
Epoch 8/10
6250/6250 [==============================] - 12s 2ms/step - loss: 0.5210
Epoch 9/10
6250/6250 [==============================] - 13s 2ms/step - loss: 0.5210
Epoch 10/10
6250/6250 [==============================] - 13s 2ms/step - loss: 0.5210


#### Updated Weights

In [13]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

W1:
 [[-0.01730815 12.850388   14.410232  ]
 [-8.939161    0.33438644 11.966936  ]] 
b1: [-11.226457   13.396328    1.7587752]
W2:
 [[-45.495697]
 [ 38.57591 ]
 [-43.28297 ]] 
b2: [-12.436532]


In [14]:
W1 = np.array([
    [-8.94,  0.29, 12.89],
    [-0.17, -7.34, 10.79]] )
b1 = np.array([-9.87, -9.28,  1.01])
W2 = np.array([
    [-31.38],
    [-27.86],
    [-32.79]])
b2 = np.array([15.54])
model.get_layer("layer1").set_weights([W1,b1])
model.get_layer("layer2").set_weights([W2,b2])

#### Predict for new data

In [15]:
X_test = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
X_testn = norm_l(X_test)
predictions = model.predict(X_testn)
print("predictions = \n", predictions)

1/1 [==============================] - 0s 125ms/step
predictions = 
 [[9.625139e-01]
 [3.031606e-08]]


In [16]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

decisions = 
[[1.]
 [0.]]


In [17]:
yhat = (predictions >= 0.5).astype(int)
print(f"decisions = \n{yhat}")

decisions = 
[[1]
 [0]]


### <a name="2.4"></a>
### 2.4 Numpy Model (Forward Prop in NumPy)

In [19]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [20]:
# Define the activation function
g = sigmoid

In [21]:
def my_dense(a_in, W, b):
    units = W.shape[1]
    a_out = np.zeros(units)
    for j in range(units):               
        w = W[:,j]                                    
        z = np.dot(w, a_in) + b[j]         
        a_out[j] = g(z)               
    return(a_out)

In [22]:
def my_sequential(x, W1, b1, W2, b2):
    a1 = my_dense(x,  W1, b1)
    a2 = my_dense(a1, W2, b2)
    return(a2)

In [23]:
W1_tmp = np.array( [[-8.93,  0.29, 12.9 ], [-0.1,  -7.32, 10.81]] )
b1_tmp = np.array( [-9.82, -9.28,  0.96] )
W2_tmp = np.array( [[-31.18], [-27.59], [-32.56]] )
b2_tmp = np.array( [15.41] )

#### Predict for new data

In [24]:
def my_predict(X, W1, b1, W2, b2):
    m = X.shape[0]
    p = np.zeros((m,1))
    for i in range(m):
        p[i,0] = my_sequential(X[i], W1, b1, W2, b2)
    return(p)

In [25]:
X_tst = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
X_tstn = norm_l(X_tst)  # remember to normalize
predictions = my_predict(X_tstn, W1_tmp, b1_tmp, W2_tmp, b2_tmp)

In [26]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

decisions = 
[[1.]
 [0.]]


In [27]:
yhat = (predictions >= 0.5).astype(int)
print(f"decisions = \n{yhat}")

decisions = 
[[1]
 [0]]
